In [1]:
import $ivy.`org.apache.spark::spark-core:3.5.6`
import $ivy.`org.apache.spark::spark-graphx:3.5.6`

import $ivy.$
import $ivy.$

In [2]:
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.log4j.{Level, Logger}

In [3]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

In [4]:
val conf = new SparkConf()
                .setAppName("GraphX")
                .setMaster("local[*]")
                .set("spark.log.level", "WARN")
                
val sc = new SparkContext(conf)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/13 14:29:20 INFO SparkContext: Running Spark version 3.5.6
26/01/13 14:29:20 INFO SparkContext: OS info Mac OS X, 26.2, aarch64
26/01/13 14:29:20 INFO SparkContext: Java version 11.0.29
26/01/13 14:29:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


conf: SparkConf = org.apache.spark.SparkConf@e2a65fc
sc: SparkContext = org.apache.spark.SparkContext@3713e0b9

# Создаём граф

In [5]:
// Create an RDD for the vertices
val users: RDD[(VertexId, (String, String))] =
  sc.parallelize(Seq((3L, ("rxin", "student")), (7L, ("jgonzal", "postdoc")),
                       (5L, ("franklin", "prof")), (2L, ("istoica", "prof")),
                       (4L, ("peter", "student"))))

// Create an RDD for edges
val relationships: RDD[Edge[String]] =
  sc.parallelize(Seq(Edge(3L, 7L, "collab"),    Edge(5L, 3L, "advisor"),
                       Edge(2L, 5L, "colleague"), Edge(5L, 7L, "pi"),
                       Edge(4L, 0L, "student"),   Edge(5L, 0L, "colleague")))

// Define a default user in case there are relationship with missing user
val defaultUser = ("John Doe", "Missing")

// Build the initial Graph
val graph = Graph(users, relationships, defaultUser)

users: RDD[(VertexId, (String, String))] = ParallelCollectionRDD[0] at parallelize at cmd5.sc:3
relationships: RDD[Edge[String]] = ParallelCollectionRDD[1] at parallelize at cmd5.sc:9
defaultUser: (String, String) = ("John Doe", "Missing")
graph: Graph[(String, String), String] = org.apache.spark.graphx.impl.GraphImpl@36edae54

# Операции с графами

## Information about the Graph

In [6]:
println(s"numEdges: ${graph.numEdges}\nnumVertices: ${graph.numVertices}")

numEdges: 6
numVertices: 6


In [7]:
graph.inDegrees.collect

res7: Array[(VertexId, Int)] = Array((0L, 2), (3L, 1), (5L, 1), (7L, 2))

In [8]:
graph.outDegrees.collect

res8: Array[(VertexId, Int)] = Array((2L, 1), (3L, 1), (4L, 1), (5L, 3))

In [9]:
graph.degrees.collect

res9: Array[(VertexId, Int)] = Array(
  (0L, 2),
  (2L, 1),
  (3L, 2),
  (4L, 1),
  (5L, 4),
  (7L, 2)
)

## Views of the graph as collections

In [10]:
graph.vertices.collect.foreach(println)

(0,(John Doe,Missing))
(2,(istoica,prof))
(3,(rxin,student))
(4,(peter,student))
(5,(franklin,prof))
(7,(jgonzal,postdoc))


In [11]:
graph.vertices.collect

res11: Array[(VertexId, (String, String))] = Array(
  (0L, ("John Doe", "Missing")),
  (2L, ("istoica", "prof")),
  (3L, ("rxin", "student")),
  (4L, ("peter", "student")),
  (5L, ("franklin", "prof")),
  (7L, ("jgonzal", "postdoc"))
)

In [12]:
graph.edges.collect.foreach(println)

Edge(3,7,collab)
Edge(5,3,advisor)
Edge(2,5,colleague)
Edge(5,7,pi)
Edge(4,0,student)
Edge(5,0,colleague)


In [13]:
graph.edges.collect

res13: Array[Edge[String]] = Array(
  Edge(3L, 7L, "collab"),
  Edge(5L, 3L, "advisor"),
  Edge(2L, 5L, "colleague"),
  Edge(5L, 7L, "pi"),
  Edge(4L, 0L, "student"),
  Edge(5L, 0L, "colleague")
)

In [14]:
graph.triplets.collect.foreach(println)

((3,(rxin,student)),(7,(jgonzal,postdoc)),collab)
((5,(franklin,prof)),(3,(rxin,student)),advisor)
((2,(istoica,prof)),(5,(franklin,prof)),colleague)
((5,(franklin,prof)),(7,(jgonzal,postdoc)),pi)
((4,(peter,student)),(0,(John Doe,Missing)),student)
((5,(franklin,prof)),(0,(John Doe,Missing)),colleague)


In [15]:
graph.triplets.collect

res15: Array[EdgeTriplet[(String, String), String]] = Array(
  Edge(3L, 7L, "collab"),
  Edge(5L, 3L, "advisor"),
  Edge(2L, 5L, "colleague"),
  Edge(5L, 7L, "pi"),
  Edge(4L, 0L, "student"),
  Edge(5L, 0L, "colleague")
)

In [16]:
graph.triplets.map(
  triplet => triplet.srcAttr._1 + " is the " + triplet.attr + " of " + triplet.dstAttr._1
).collect.foreach(println(_))

rxin is the collab of jgonzal
franklin is the advisor of rxin
istoica is the colleague of franklin
franklin is the pi of jgonzal
peter is the student of John Doe
franklin is the colleague of John Doe


## Structural Operators

In [17]:
// Remove missing vertices as well as the edges to connected to them
val validGraph = graph.subgraph(vpred = (id, attr) => attr._2 != "Missing")

validGraph: Graph[(String, String), String] = org.apache.spark.graphx.impl.GraphImpl@1fa82dd9

In [18]:
// The valid subgraph will disconnect users 4 and 5 by removing user 0
validGraph.vertices.collect.foreach(println(_))

(2,(istoica,prof))
(3,(rxin,student))
(4,(peter,student))
(5,(franklin,prof))
(7,(jgonzal,postdoc))


In [19]:
validGraph.triplets.map(
  triplet => triplet.srcAttr._1 + " is the " + triplet.attr + " of " + triplet.dstAttr._1
).collect.foreach(println(_))

rxin is the collab of jgonzal
franklin is the advisor of rxin
istoica is the colleague of franklin
franklin is the pi of jgonzal


In [20]:
import org.apache.spark.graphx.EdgeDirection._

graph.collectNeighborIds(In).collect()

import org.apache.spark.graphx.EdgeDirection._
res20_1: Array[(VertexId, Array[VertexId])] = Array(
  (0L, Array(4L, 5L)),
  (2L, Array()),
  (3L, Array(5L)),
  (4L, Array()),
  (5L, Array(2L)),
  (7L, Array(3L, 5L))
)

# Graph Algorithms

## PageRank

GraphX also includes an example social network dataset that we can run PageRank on. A set of users is given in ``data/graphx/users.txt``, and a set of relationships between users is given in ``data/graphx/followers.txt``. We compute the PageRank of each user as follows:

In [21]:
import org.apache.spark.graphx.GraphLoader

// Load the edges as a graph
val graph = GraphLoader.edgeListFile(sc, "/opt/spark/data/graphx/followers.txt")

// Run PageRank
val ranks = graph.pageRank(0.0001).vertices

// Join the ranks with the usernames
val users = sc.textFile("/opt/spark/data/graphx/users.txt").map { line =>
  val fields = line.split(",")
  (fields(0).toLong, fields(1))
}
val ranksByUsername = users.join(ranks).map {
  case (id, (username, rank)) => (username, rank)
}

// Print the result
println(ranksByUsername.collect().mkString("\n"))

26/01/13 14:29:27 WARN BlockManager: Block rdd_267_1 already exists on this machine; not re-adding it
26/01/13 14:29:27 WARN BlockManager: Block rdd_347_1 already exists on this machine; not re-adding it
26/01/13 14:29:27 WARN BlockManager: Block rdd_363_1 already exists on this machine; not re-adding it


(justinbieber,0.15007622780470478)
(matei_zaharia,0.7017164142469724)
(ladygaga,1.3907556008752426)
(BarackObama,1.4596227918476916)
(jeresig,0.9998520559494657)
(odersky,1.2979769092759237)


import org.apache.spark.graphx.GraphLoader
graph: Graph[Int, Int] = org.apache.spark.graphx.impl.GraphImpl@3dc71223
ranks: VertexRDD[Double] = VertexRDDImpl[900] at RDD at VertexRDD.scala:57
users: RDD[(Long, String)] = MapPartitionsRDD[909] at map at cmd21.sc:10
ranksByUsername: RDD[(String, Double)] = MapPartitionsRDD[913] at map at cmd21.sc:14

## Connected Components

GraphX contains an implementation of the algorithm in the ConnectedComponents object, and we compute the connected components of the example social network dataset from the PageRank section as follows:

In [22]:
import org.apache.spark.graphx.GraphLoader

// Load the graph as in the PageRank example
val graph = GraphLoader.edgeListFile(sc, "/opt/spark/data/graphx/followers.txt")

// Find the connected components
val cc = graph.connectedComponents().vertices

// Join the connected components with the usernames
val users = sc.textFile("/opt/spark/data/graphx/users.txt").map { line =>
  val fields = line.split(",")
  (fields(0).toLong, fields(1))
}
val ccByUsername = users.join(cc).map {
  case (id, (username, cc)) => (username, cc)
}

// Print the result
println(ccByUsername.collect().mkString("\n"))

26/01/13 14:29:30 WARN BlockManager: Block rdd_925_0 already exists on this machine; not re-adding it


(justinbieber,1)
(matei_zaharia,3)
(ladygaga,1)
(BarackObama,1)
(jeresig,3)
(odersky,3)


import org.apache.spark.graphx.GraphLoader
graph: Graph[Int, Int] = org.apache.spark.graphx.impl.GraphImpl@598a14cc
cc: VertexRDD[VertexId] = VertexRDDImpl[946] at RDD at VertexRDD.scala:57
users: RDD[(Long, String)] = MapPartitionsRDD[963] at map at cmd22.sc:10
ccByUsername: RDD[(String, VertexId)] = MapPartitionsRDD[967] at map at cmd22.sc:14

## Triangle Counting

We compute the triangle count of the social network dataset from the PageRank section. Note that TriangleCount requires the edges to be in canonical orientation (srcId < dstId) and the graph to be partitioned using Graph.partitionBy.

In [23]:
import org.apache.spark.graphx.{GraphLoader, PartitionStrategy}

// Load the edges in canonical order and partition the graph for triangle count
val graph = GraphLoader.edgeListFile(sc, "/opt/spark/data/graphx/followers.txt", true)
  .partitionBy(PartitionStrategy.RandomVertexCut)
  
// Find the triangle count for each vertex
val triCounts = graph.triangleCount().vertices

// Join the triangle counts with the usernames
val users = sc.textFile("/opt/spark/data/graphx/users.txt").map { line =>
  val fields = line.split(",")
  (fields(0).toLong, fields(1))
}
val triCountByUsername = users.join(triCounts).map { 
  case (id, (username, tc)) => (username, tc)
}

// Print the result
println(triCountByUsername.collect().mkString("\n"))

26/01/13 14:29:30 WARN ShippableVertexPartitionOps: Joining two VertexPartitions with different indexes is slow.
26/01/13 14:29:30 WARN ShippableVertexPartitionOps: Joining two VertexPartitions with different indexes is slow.


(justinbieber,0)
(matei_zaharia,1)
(ladygaga,0)
(BarackObama,0)
(jeresig,1)
(odersky,1)


import org.apache.spark.graphx.{GraphLoader, PartitionStrategy}
graph: Graph[Int, Int] = org.apache.spark.graphx.impl.GraphImpl@6b798187
triCounts: VertexRDD[Int] = VertexRDDImpl[1037] at RDD at VertexRDD.scala:57
users: RDD[(Long, String)] = MapPartitionsRDD[1042] at map at cmd23.sc:11
triCountByUsername: RDD[(String, Int)] = MapPartitionsRDD[1046] at map at cmd23.sc:15

# Pregel

We can use the Pregel operator to express computation such as single source shortest path in the following example.

In [24]:
import org.apache.spark.graphx.{Graph, VertexId}
import org.apache.spark.graphx.util.GraphGenerators

// A graph with edge attributes containing distances
val graph: Graph[Long, Double] = GraphGenerators.logNormalGraph(sc, numVertices = 100).mapEdges(e => e.attr.toDouble)

val sourceId: VertexId = 42 // The ultimate source

// Initialize the graph such that all vertices except the root have distance infinity.
val initialGraph = graph.mapVertices((id, _) => if (id == sourceId) 0.0 else Double.PositiveInfinity)

val sssp = initialGraph.pregel(Double.PositiveInfinity)(
  (id, dist, newDist) => math.min(dist, newDist), // Vertex Program
  triplet => {  // Send Message
    if (triplet.srcAttr + triplet.attr < triplet.dstAttr) {
      Iterator((triplet.dstId, triplet.srcAttr + triplet.attr))
    } else {
      Iterator.empty
    }
  },
  (a, b) => math.min(a, b) // Merge Message
)

println(sssp.vertices.collect.mkString("\n"))

(96,2.0)
(56,2.0)
(16,2.0)
(80,2.0)
(48,2.0)
(32,2.0)
(0,2.0)
(24,2.0)
(64,2.0)
(40,2.0)
(72,2.0)
(8,1.0)
(88,2.0)
(41,1.0)
(81,2.0)
(97,2.0)
(25,2.0)
(65,2.0)
(73,2.0)
(57,2.0)
(33,2.0)
(1,2.0)
(89,2.0)
(17,2.0)
(9,2.0)
(49,2.0)
(34,2.0)
(82,1.0)
(66,2.0)
(98,2.0)
(50,2.0)
(42,0.0)
(74,2.0)
(90,1.0)
(18,1.0)
(58,2.0)
(26,2.0)
(10,2.0)
(2,2.0)
(19,2.0)
(59,2.0)
(11,2.0)
(35,1.0)
(27,2.0)
(75,2.0)
(51,2.0)
(83,1.0)
(67,2.0)
(3,2.0)
(91,1.0)
(43,2.0)
(99,2.0)
(84,2.0)
(52,2.0)
(4,2.0)
(76,1.0)
(28,2.0)
(36,2.0)
(92,2.0)
(20,2.0)
(12,2.0)
(60,2.0)
(44,1.0)
(68,2.0)
(13,2.0)
(61,2.0)
(21,2.0)
(77,2.0)
(53,2.0)
(29,2.0)
(93,2.0)
(37,1.0)
(45,2.0)
(69,1.0)
(85,2.0)
(5,2.0)
(22,2.0)
(54,2.0)
(46,2.0)
(30,2.0)
(14,2.0)
(62,2.0)
(6,2.0)
(70,2.0)
(38,2.0)
(86,2.0)
(78,2.0)
(94,1.0)
(39,2.0)
(15,2.0)
(47,1.0)
(71,2.0)
(55,2.0)
(95,2.0)
(79,2.0)
(23,2.0)
(63,2.0)
(7,2.0)
(31,2.0)
(87,2.0)


import org.apache.spark.graphx.{Graph, VertexId}
import org.apache.spark.graphx.util.GraphGenerators
graph: Graph[Long, Double] = org.apache.spark.graphx.impl.GraphImpl@59bdbf0c
sourceId: VertexId = 42L
initialGraph: Graph[Double, Double] = org.apache.spark.graphx.impl.GraphImpl@5f655399
sssp: Graph[Double, Double] = org.apache.spark.graphx.impl.GraphImpl@7744eeff